<a id= 'top'></a>

# Visualization and initial exploration of data:
<p>
In this notebook, I have looked in to the two json files that contain metadata for the recordings and decided on the data set that will be explored in the rest of the codes. 
<p>

1. [Reading and processing the json files](#1)
* [Merging the datasets](#2)
* [Number of companies](#3)
* [Quality score](#4)
* [Making categorical dataset](#5)
* [Call length](#6)
* [Under sampling](#7)

In [1]:
## to have a cleaner notebook I will import the required libraries and functions from the attached utilities.py file. 
from utilities import *
%load_ext watermark
%watermark -a 'Sohrab Redjai Sani' -u -d -v -p numpy,pandas,matplotlib

plotly  1.13.0
Sohrab Redjai Sani 
last updated: 2017-09-30 

CPython 3.5.2
IPython 5.1.0

numpy 1.13.1
pandas 0.18.1
matplotlib 1.5.3


<p>
<a id= '1'></a>

## Reading and processing the json files:
<p>
[Return to top](#top)

In [2]:
df = pd.read_json('/Users/srs/Dropbox/GitHub/MaestroQA/mfcc/voice_comments.json') ## S3 link (recording_url)
df2 = pd.read_json('/Users/srs/Dropbox/GitHub/MaestroQA/mfcc/voice_gradables.json') ## score (qa_score)

In [3]:
df.columns

Index(['_id', 'author_type', 'body', 'call_type', 'comment_type',
       'composer_id', 'gradable_id', 'length', 'public', 'recording_url',
       'time'],
      dtype='object')

In [4]:
df2.columns

Index(['_id', 'composer_id', 'csat_score', 'first_seen', 'gradable_id',
       'qa_score', 'scoreType', 'subject', 'tags'],
      dtype='object')

<p>
<a id= '1'></a>

## Merging the datasets:
<p>
[Return to top](#top)

In [5]:
## makeing a dataset from the df with and droping NAN, sorting on "gradable_id" and droping duplicates
df3 = df[["gradable_id","recording_url","length","comment_type","composer_id","time","public"]].dropna().reset_index().sort('gradable_id', ascending=True).drop_duplicates(subset = "gradable_id")
## makeing a dataset from the df2, sorting on "gradable_id" and droping duplicates
df4 = df2[["gradable_id","qa_score", "tags","csat_score"]].sort('gradable_id', ascending=True).drop_duplicates(subset = "gradable_id")
## df5 is the merge of df3 and df4 on "gradable_id".
## basically this dataset has all the scored recording and their S3 links from df.
df_total = df3.merge(df4, on="gradable_id")
df_total.shape

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



(10729, 11)

<p>
<a id= '2'></a>

## Number of companies:
<p>

In this dataset, we recordings from 16 companies, with six of them have more than 500 recordings.   
<p>
[Return to top](#top)

In [6]:
qa = df_total.composer_id.value_counts()
print ('number of companies:', qa.shape)
iplot(qa.iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='Number of call',
                 title='Company name',
              ))

number of companies: (16,)


<p>
<a id= '4'></a>

## Quality score:
<p>

The following image shows the number of calls vs their quality. This image identify the very skewed and unbalanced nature of these calls.  

<p>
[Return to top](#top)


In [7]:
qa = df_total.qa_score.value_counts()

iplot(qa.iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='Number of call',
                 title='Quality score',
                   ))

<p>
<a id= '5'></a>

## Making categorical dataset: 

<p>

After talking with experts in the field (call support managers), they told us that they would be interested to hear the calls that get scored less than 90 as they are considered low quality interaction. Therefore, I decided to separate the calls in to two groups. Good and bad calls with score larger than 90 and score less than 90 respectively. 
<p>
[Return to top](#top)



In [8]:
df_total['category'] = df_total['qa_score'].apply(category)

In [9]:
qa = df_total.category.value_counts()

iplot(qa.iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='Number of call',
                 title='Quality score',
                   ))

<p>
<a id= '6'></a>

## Call length:
<p>

The following image shows the length of the distribution of length of the calls. As most of the calls are see that 10 minute. I removed the call longer than for computational prepossess. 
<p>
Also from the following images we can see that there is no obvious bias for a call to be good or bad as the function of it’s length. 


<p>
[Return to top](#top)



In [10]:
df_total['length_min'] = df_total.length.apply(sectomin)

qa = df_total.length_min.value_counts()

## remove all the calls that is longer than 600 sec

df_shortcalls = df_total[df_total.length <= 600]
df_shortcalls.shape


(9949, 13)

In [11]:
qa = df_shortcalls.length_min.value_counts()
iplot(qa.iplot(asFigure=True,
                 kind='bar',
                 xTitle=' ',
                 yTitle='Number of call',
                 title='Call lenght' ))

### Looking at the distribuiton of good and bad calls as the function of call length:

In [12]:
df_bad = df_shortcalls[df_shortcalls.category == 0]
df_good = df_shortcalls[df_shortcalls.category == 1]

In [13]:
df_bad.length_min.describe()

count    2881.000000
mean        4.195326
std         1.774852
min         0.400000
25%         2.950000
50%         3.833333
75%         5.083333
max         9.983333
Name: length_min, dtype: float64

In [14]:
df_good.length_min.describe()

count    7068.000000
mean        4.116226
std         1.755476
min         0.166667
25%         2.916667
50%         3.716667
75%         4.983333
max        10.000000
Name: length_min, dtype: float64

In [15]:
qa = df_bad.length_min.value_counts()
iplot(qa.iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='Number of call',
                 title='Length of the Bad Calls in minutes'))

In [16]:
qa = df_good.length_min.value_counts()
iplot(qa.iplot(asFigure=True,
                 kind='bar',
                 xTitle='',
                 yTitle='Number of call',
                 title='Length of the Bad Calls in minutes'))

<p>
<a id= '7'></a>

## Under sampling:
 
<p>
Afterwards, to make a balanced dataset for future classification, I manual under sampled the dataset by only keeps 400 of the calls with score 100 out of the 5023 calls. 
<p>
So at the end we have two balanced categories of good(1) and bad(0) with each of them containing 2912 calls. 

<p>
[Return to top](#top)

In [17]:
df_no100 = df_shortcalls[df_shortcalls.qa_score != 100]
df_total_100 = df_shortcalls [df_shortcalls.qa_score == 100]
df_total_100_400 = df_shortcalls[:430].copy()

df_balanced = df_no100.append(df_total_100_400)
df_balanced.shape

(5824, 13)

In [18]:
qa = df_balanced.category.value_counts()
iplot(qa.iplot(asFigure=True,
                 kind='bar',
                 xTitle='Quality score',
                 yTitle='Number of call',
                 title='Quality score' ))

In [19]:
df_balanced.to_csv('metadata.csv')